# Importing Packages

In [1]:
import os
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

In [2]:
from unet import build_unet
from customMetrics import dice_loss, dice_coef, iou

# Setting The Environment

In [3]:
H = 128
W = 128

In [4]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Hyperparameters """
batch_size = 1
lr = 1e-4
num_epochs = 100
model_path = os.path.join("files128", "model128.h5")
csv_path = os.path.join("files128", "data.csv")

In [5]:
""" Dataset """
dataset_path = "D:\\new_data_128"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "valid")

# Utils Functions

In [6]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [7]:
def load_data(path):
    print(path)
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "gt", "*.jpg")))
    return x, y

In [8]:
def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

In [9]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

In [10]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

In [11]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y


In [12]:
def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

# Loading The Dataset

In [13]:
train_x, train_y = load_data(train_path)
# train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")


D:\new_data_128\train
D:\new_data_128\valid
Train: 1890 - 1890
Valid: 630 - 630


In [14]:
""" Directory to save files """
create_dir("files128")

In [15]:
train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_setps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_setps += 1

# The Model

In [16]:
""" Model """
model = build_unet((H, W, 3))
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 64) 0           batch_normalization[0][0]        
_______________________________________________________________________________________________

## Callbacks

In [17]:
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
]

In [ ]:
model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_setps,
    callbacks=callbacks
)

Epoch 1/100
1890/1890 [==============================] - 285s 136ms/step - loss: 0.3661 - dice_coef: 0.6339 - iou: 0.4959 - recall: 0.4545 - precision: 0.7203 - val_loss: 0.4593 - val_dice_coef: 0.5407 - val_iou: 0.3991 - val_recall: 0.2740 - val_precision: 0.9218

Epoch 00001: val_loss improved from inf to 0.45934, saving model to files128\model128.h5
Epoch 2/100
1890/1890 [==============================] - 252s 133ms/step - loss: 0.2308 - dice_coef: 0.7692 - iou: 0.6392 - recall: 0.4398 - precision: 0.8866 - val_loss: 0.4230 - val_dice_coef: 0.5770 - val_iou: 0.4348 - val_recall: 0.2762 - val_precision: 0.9420

Epoch 00002: val_loss improved from 0.45934 to 0.42302, saving model to files128\model128.h5
Epoch 3/100
1890/1890 [==============================] - 250s 132ms/step - loss: 0.2002 - dice_coef: 0.7998 - iou: 0.6781 - recall: 0.4478 - precision: 0.9020 - val_loss: 0.3992 - val_dice_coef: 0.6008 - val_iou: 0.4587 - val_recall: 0.2998 - val_precision: 0.9351

Epoch 00003: val_los

In [26]:
def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)